## Imports

In [14]:
import copy
import json
import math
import pprint
import random
import scipy
import sklearn.model_selection
import statistics

## Data loading and processing

In [2]:
ranked_data_path = f"bigresponse.json"
ranked_data_file = open(ranked_data_path,encoding="utf-8")
ranked_data = json.load(ranked_data_file)

In [3]:
players = ranked_data["players"]
maps = ranked_data["maps"]
scores = ranked_data["scores"]
filtered_scores = [score for score in scores if score["accuracy"] < 100]

In [4]:
players_by_id = {player["id"] : player for player in players}
maps_by_id = {bmap["id"] : bmap for bmap in maps}

In [5]:
scores_by_player_id = {}
scores_by_map_id = {}

def add_score(score):
    player_id = score["playerId"]
    map_id = score["leaderboardId"]
    
    player_map = scores_by_player_id.get(player_id,{})
    player_map[map_id] = score
    
    scores_by_player_id[player_id] = player_map
    
    map_map = scores_by_map_id.get(map_id,{})
    map_map[player_id] = score
    
    scores_by_map_id[map_id] = map_map
    
#for score in scores:
for score in filtered_scores:
    add_score(score)

In [6]:
modifiers = {"SF":1.2,
             "GN":1,
             "SA":1.05,
             "PM":1,
             "IF":1,
             "NO":0.5,
             "BE":1,
             "SS":0.75,
             "FS":1.05,
             "NB":0.5,
             "SC":1,
             "OD":1,
             "DA":1,
             "CS":1,
             "NA":0.5,
             "OP":0.5}
"""
"SF" - Super Fast,
"GN" - Ghost Notes,
"SA" - Strict Angles,
"PM" - Pro Mode,
"IF" - 1 life,
"NO" - No obstacles (no walls),
"BE" - Battery Energy,
"SS" - Slow song,
"FS" - Fast song,
"NB" - No bombs,
"SC" - Small Notes,
"OD" - Old Dots,
"DA" - Disappearing Arrows,
"CS" - ???,
"NA" - No Arrows -50%,
"OP" - Out of platform -50% 
"""

def modified_score(score,score_modifiers):
    final_score = score
    for modifier,multiplier in modifiers.items():
        if modifier in score_modifiers:
            if multiplier > 1:
                final_score = 1 - (1 - final_score)*(2 - multiplier)
            else:
                final_score = final_score * multiplier
            
    return final_score

## Mathematical functions for transforming and combining numerical values

In [8]:
"""
This is a mapping on the probability space rather than the population space.
That is, from (0,1) to (0,1). A probability distribution on the (0,1) interval.
We can do this with the Beta distribution, though other distributions might work.

I approximate a value for alpha and beta using this: https://homepage.divms.uiowa.edu/~mbognar/applets/beta.html
Trying to get the median around 0.9 and the probability close to 0 for 0.5, but a little bit higher for 0.65ish.
Beat Saber scores are typically at least 65%, on median probably around 0.9. They also must go down as the value
approaches 1, so beta must be greater than 1.

alpha = 10 and beta = 1.25 seemed to work quite well.
"""
beta_alpha = 10
beta_beta = 1.25

def beta_value(perc_value, beta_alpha=beta_alpha, beta_beta=beta_beta):
    return scipy.stats.beta.cdf(perc_value, beta_alpha, beta_beta)

def perc_beta_value(beta_value,beta_alpha=beta_alpha, beta_beta=beta_beta):
    return scipy.stats.beta.ppf(beta_value, beta_alpha, beta_beta)

In [9]:
"""
There are some issues with this, like that a 100% score could lower a player's skill level
if it is on a very easy map. But this is very unlikely with the numbers, and 100% scores
are exceedingly rare anyway. There are also ways to deal with that later after estimating
difficulties.
"""

# truncexp_max = 25
truncexp_max = 100
# truncexp_base_mean = 7
truncexp_base_mean = 10

def truncexp_value(perc_value,base_mean=truncexp_base_mean,maxx=truncexp_max,beta_alpha=beta_alpha,beta_beta=beta_beta):
    return scipy.stats.truncexpon.ppf(beta_value(perc_value,beta_alpha=beta_alpha,beta_beta=beta_beta),b=maxx,scale=base_mean)

def perc_truncexp_value(truncexp_value,base_mean=truncexp_base_mean,maxx=truncexp_max,beta_alpha=beta_alpha,beta_beta=beta_beta):
    return perc_beta_value(scipy.stats.truncexpon.cdf(truncexp_value,b=maxx,scale=base_mean),beta_alpha=beta_alpha,beta_beta=beta_beta)

In [20]:
"""
We normalize around a value of 7.
This means that a X star player on a 7 accability map achieves an X star score
and a 7 star player on an X accability map achieves an X star score.
"""

#linear_mean = 7
linear_mean = 10

def score_linear(skill,accability,linear_mean = linear_mean):
    return skill * accability / linear_mean

def accability_linear(skill,score,linear_mean = linear_mean):
    return score / skill * linear_mean

def skill_linear(accability,score,linear_mean = linear_mean):
    return score / accability * linear_mean

In [21]:
def abs_prop_error(value,evalue):        
    return abs((value-evalue)/evalue)

In [22]:
"""
Let's start with something simple: Just use the median instead of the average.
"""

def aggregation_median(scores,default_value=1):
    if scores == []:
        return default_value
    else:
        return statistics.median(scores)
    
def aggregation_median_f(default_value):
    def f(scores):
        return aggregation_median(scores,default_value)
    
    return f

In [23]:
"""
Do the average of the top percentage of scores.
"""
def aggregation_topscores(scores,perc=0.25,default_value=1):
    scores.sort()
    l = len(scores)
    n = math.floor(l*perc)
    
    if n == 0:
        return default_value
    else:
        return statistics.mean(scores[l-n:])
    
def aggregation_topscores_f(perc,default_value):
    def f(scores):
        return aggregation_topscores(scores,perc,default_value)
    
    return f

In [24]:
"""
Do the average of the bottom percentage of scores.
"""
def aggregation_bottomscores(scores,perc=0.25,default_value=1):
    scores.sort()
    l = len(scores)
    n = math.floor(l*perc)
    
    if n == 0:
        return default_value
    else:
        return statistics.mean(scores[0:n])
    
def aggregation_bottomscores_f(perc,default_value):
    def f(scores):
        return aggregation_bottomscores(scores,perc,default_value)
    
    return f

## Main algorithm

In [25]:
class BiPartiteStabilizer:
    """No description yet"""
    
    def __init__(self,
                 anodes_ratings,bnodes_ratings,
                 wdata,afun,bfun,wfun,
                 aggregation_fun=aggregation_median,
                 default_rating=1,
                 max_iter=50,
                 error_fun=abs_prop_error,
                 error_aggregation_fun=aggregation_bottomscores,
                 finish_early=True,
                 error_change_prop=0.001):
        """
        anodes_ratings and bnodes_ratings must be dictionaries with the node identifiers as keys
        and initial ratings as values.
        
        wdata must be a doubly indexed dictionary with anode identifiers and bnode identifiers as respective indexes,
        respectively, and weights as values.
        
        afun and bfun must be functions taking a value of the other node type as first argument
        and a weight as the second argument, that returns the corresponding value for the node.
        
        Similarly, wfun must be a function that takes the value of an anode and bnode and returns the
        correct weight.
        
        These functions must be such that:
        - wfun(afun(b,w),b) = w
        - wfun(a,bfun(a,w)) = w
        - afun(bfun(a,w),w) = a
        - afun(b,wfun(a,b)) = a
        - bfun(afun(b,w),w) = b
        - bfun(a,wfun(a,b)) = b
        
        error_fun must take two parameters (actual value, expected value) and return a number indicating the error for that
        particular edge
        """
        self.anodes_ratings = anodes_ratings
        self.bnodes_ratings = bnodes_ratings
        
        self.adata = wdata
        self.bdata = self.process_bdata(wdata)
        
        self.afun = afun
        self.bfun = bfun
        self.wfun = wfun
        
        self.aggregation_fun = aggregation_fun
        
        self.error_fun = error_fun
        self.error_aggregation_fun = error_aggregation_fun
        
        self.default_rating = default_rating
        
        self.average_error = math.inf
        
        self.iter = 0
        self.max_iter = max_iter
        
        # Finish early when error increases
        self.finish_early = finish_early
        self.last_average_error = math.inf
        self.last_anodes_ratings = anodes_ratings
        self.last_bnodes_ratings = bnodes_ratings       
    
        self.error_change_prop = error_change_prop
    
    def process_bdata(self, wdata):
        bdata = {}
        
        for anode_id, anode_data in wdata.items():
            for bnode_id, w in anode_data.items():
                bnode_data = bdata.get(bnode_id,{})
                bnode_data[anode_id] = w
                bdata[bnode_id] = bnode_data
        
        return bdata       
        
    def acycle(self):
        calc_values = []
        for anode_id in self.anodes_ratings:
            if anode_id in self.adata:
                calc_values.append(self.anode_process(anode_id))
            
        self.average_error = self.error_aggregation_fun(calc_values)
    
    def anode_process(self,anode_id):
        anode_data = self.adata.get(anode_id,False)
                
        calc_values = []
        for bnode_id, w in anode_data.items():
            bnode_value = self.bnodes_ratings.get(bnode_id,self.default_rating)
            # asum += clamp(self.afun(bnode_value,w))
            # asum += self.afun(bnode_value,w)
            calc_values.append(self.afun(bnode_value,w))
        
        avg = self.aggregation_fun(calc_values)
        
        # self.anodes_ratings[anode_id] = clamp(avg)
        self.anodes_ratings[anode_id] = avg
                                              
        return self.anode_error(anode_id)
        
    def anode_error(self,anode_id):
        anode_data = self.adata[anode_id]
        anode_value = self.anodes_ratings.get(anode_id,self.default_rating)
        
        calc_values = []
        for bnode_id, w in anode_data.items():
            bnode_value = self.bnodes_ratings.get(bnode_id,self.default_rating)
            # calculated_w = clamp(self.wfun(anode_value,bnode_value))
            calculated_w = self.wfun(anode_value,bnode_value)
            # error = clamp(self.error_fun(calculated_w,w))
            error = self.error_fun(calculated_w,w)
            calc_values.append(error)
            
        return self.error_aggregation_fun(calc_values)
    
    def bcycle(self):
        for bnode_id in self.bnodes_ratings:
            if bnode_id in self.bdata:
                self.bnode_process(bnode_id)                
    
    def bnode_process(self,bnode_id):
        bnode_data = self.bdata[bnode_id]
        
        calc_values = []
        for anode_id, w in bnode_data.items():
            anode_value = self.anodes_ratings.get(anode_id,self.default_rating)
            # bsum += clamp(self.bfun(anode_value,w))
            calc_values.append(self.bfun(anode_value,w))
        
        avg = self.aggregation_fun(calc_values)
        
        # self.bnodes_ratings[bnode_id] = clamp(avg)
        self.bnodes_ratings[bnode_id] = avg
    
    def save_last(self):
        self.last_anodes_ratings = self.anodes_ratings
        self.last_bnodes_ratings = self.bnodes_ratings
        self.last_average_error = self.average_error
        
    def restore_last(self):
        self.anodes_ratings = self.last_anodes_ratings
        self.bnodes_ratings = self.last_bnodes_ratings
        self.average_error = self.last_average_error
    
    def iterate(self):
        self.save_last()
        
        self.bcycle()
        self.acycle()
        
        self.iter += 1        
        print(f"Iteration {self.iter}, Average error: {self.average_error}")         
        
    def run(self):
        while (self.iter <= self.max_iter):            
            self.iterate()
            
            # Finish early with previous result if error went up
            if self.finish_early and (self.average_error > self.last_average_error):
                self.restore_last()
                print(f"Finishing early due to increased average error. Restoring previous values.")
                break
                
            # Finish if the error did not change by more than the error_change_prop
            if self.last_average_error != math.inf:
                abs_error_change = abs(self.average_error - self.last_average_error)
                if (abs_error_change / self.last_average_error) < self.error_change_prop:
                    print(f"Finishing due to change in average error less than {self.error_change_prop} (proportional)")
                    break
                
        return (self.anodes_ratings,self.bnodes_ratings)
    
    def measure_error(self,wdata_test):
        calc_values = []
        for anode_id in wdata_test:
            calc_values.append(self.test_node_error(wdata_test,anode_id))
            
        test_average_error = self.error_aggregation_fun(calc_values)
        
        return test_average_error
        
    def test_node_error(self,wdata_test,anode_id):
        anode_data = wdata_test[anode_id]
        anode_value = self.anodes_ratings.get(anode_id,self.default_rating)
        
        calc_values = []
        for bnode_id, w in anode_data.items():
            bnode_value = self.bnodes_ratings.get(bnode_id,self.default_rating)
            # calculated_w = clamp(self.wfun(anode_value,bnode_value))
            calculated_w = self.wfun(anode_value,bnode_value)
            # error = clamp(self.error_fun(calculated_w,w))
            error = self.error_fun(calculated_w,w)
            calc_values.append(error)
            
        return self.error_aggregation_fun(calc_values)     

## Prepare scores in the right format

In [39]:
scores_doubly_indexed = {map_id : {player_id : truncexp_value(modified_score(scores_by_map_id[map_id][player_id]["accuracy"],scores_by_map_id[map_id][player_id]["modifiers"])) for player_id in scores_by_map_id[map_id]} for map_id in scores_by_map_id}

In [41]:
test_set_size = 0.2

scores_doubly_indexed_training_items, scores_doubly_indexed_test_items = sklearn.model_selection.train_test_split(list(scores_doubly_indexed.items()),test_size=test_set_size)
scores_doubly_indexed_training = dict(scores_doubly_indexed_training_items)
scores_doubly_indexed_test = dict(scores_doubly_indexed_test_items)

## Run the algorithm

In [42]:
#default_rating = 7
default_rating = 10

map_ratings = {map_id : default_rating for map_id in scores_by_map_id}
player_ratings = {player_id: default_rating for player_id in scores_by_player_id}

# aggregation_fun = aggregation_median_f(default_rating)
aggregation_topscores_p = 0.75
error_aggregation_bottomscores_p = 0.75
aggregation_fun = aggregation_topscores_f(aggregation_topscores_p,default_rating)
error_aggregation_fun=aggregation_bottomscores_f(error_aggregation_bottomscores_p,0)

#bps = BiPartiteStabilizer(map_ratings,player_ratings,scores_doubly_indexed,
bps = BiPartiteStabilizer(map_ratings,player_ratings,scores_doubly_indexed_training,
                          accability_linear,skill_linear,score_linear,
                          aggregation_fun=aggregation_fun,
                          default_rating = default_rating,
                          error_aggregation_fun=error_aggregation_fun,
                          max_iter=50,
                          finish_early=False,
                          error_change_prop=0.001)

#bps.iterate()
#(maps_1,players_1) = (copy.deepcopy(bps.anodes_ratings), copy.deepcopy(bps.bnodes_ratings))
#maps_1_errors = {key:bps.anode_error(key) for key in maps_1}
#bps.iterate()
#(maps_2,players_2) = (copy.deepcopy(bps.anodes_ratings), copy.deepcopy(bps.bnodes_ratings))

(map_ratings,player_ratings) = bps.run()

Iteration 1, Average error: 0.20613466477063178
Iteration 2, Average error: 0.18162206364327296
Iteration 3, Average error: 0.18014452472697165
Iteration 4, Average error: 0.18170428569928537
Iteration 5, Average error: 0.18325685122745455
Iteration 6, Average error: 0.18437387491622761
Iteration 7, Average error: 0.18510074139212887
Iteration 8, Average error: 0.18556189067318432
Iteration 9, Average error: 0.18585036952051526
Iteration 10, Average error: 0.18602932331034777
Finishing due to change in average error less than 0.001 (proportional)


## Basic measurement of error

In [43]:
print(bps.measure_error(scores_doubly_indexed_test))

0.29425278278857353


## Hyper-parameter search

In [31]:
# Reset the hyper-parameter search result list
# Problem variations. Each of these will have a best result. But they need compared manually.
problem_grid = {
    "error_aggregation_bottomscores_p": [0.25,0.5,0.75,0.9]
}

hs_results = []
for hs_error_aggregation_bottomscores_p in problem_grid["error_aggregation_bottomscores_p"]:
    hs_results.append({
        "problem_params":{"error_aggregation_bottomscores_p":hs_error_aggregation_bottomscores_p},
        "runs":[]
    })

In [33]:
# Fixed parameters
default_rating = 10
truncexp_base_mean = default_rating
linear_mean = default_rating
test_set_size = 0.2
crossvalidation_splits = 5

# Hyperparameter grid. Some may only have one value but in principle it makes sense to explore more.
hyperparam_grid = {
    "beta_alpha": [10],
    "beta_beta": [1.25],
    "truncexp_max": [100],
    "aggregation_topscores_p": [0.1,0.25,0.5,0.75],    
    "max_iter":[50],
    "finish_early": [True,False],
    "error_change_prop": [0.005,0.001,0.0002]    
}

def hyperparam_grid_size(hyperparam_grid):
    result = 1
    for key,values in hyperparam_grid.items():
        result = result*len(values)
        
    return result

print(hyperparam_grid_size(hyperparam_grid))

# This hyper-parameter search is performed more or less manually, for two reasons:
# - It is considerably easier to program and make sure it works.
# - It allows us to optimize some of the operations to not have to re-do them when the parameters that affect it
#  have not changed (e.g. truncexp_max affecting data pre-processing)

for hs_error_aggregation_bottomscores_p in problem_grid["error_aggregation_bottomscores_p"]:
    print(f"error_aggregation_bottomscores_p = {hs_error_aggregation_bottomscores_p}")
    
    for hs_beta_alpha in hyperparam_grid["beta_alpha"]:
        print(f"beta-alpha = {hs_beta_alpha}")
        for hs_beta_beta in hyperparam_grid["beta_beta"]:
            print(f"beta-beta = {hs_beta_beta}")
            hs_beta_value = lambda perc_value : beta_value(perc_value,beta_alpha=hs_beta_alpha,beta_beta=hs_beta_beta)
            hs_perc_beta_value = lambda beta_value : perc_beta_value(beta_value,beta_alpha=hs_beta_alpha,beta_beta=hs_beta_beta)

            for hs_truncexp_max in hyperparam_grid["truncexp_max"]:
                print(f"truncexp_max = {hs_truncexp_max}")
                hs_truncexp_value = lambda perc_value : truncexp_value(perc_value,base_mean=truncexp_base_mean,maxx=hs_truncexp_max,beta_alpha=hs_beta_alpha,beta_beta=hs_beta_beta)
                hs_perc_truncexp_value = lambda truncexp_value : perc_truncexp_value(truncexp_value,base_mean=truncexp_base_mean,maxx=hs_truncexp_max,beta_alpha=hs_beta_alpha,beta_beta=hs_beta_beta)

                hs_scores_doubly_indexed = {map_id : {player_id : hs_truncexp_value(modified_score(scores_by_map_id[map_id][player_id]["accuracy"],scores_by_map_id[map_id][player_id]["modifiers"])) for player_id in scores_by_map_id[map_id]} for map_id in scores_by_map_id}

                hs_score_linear = lambda skill, accability : score_linear(skill,accability,linear_mean = linear_mean)
                hs_accability_linear = lambda skill, score : accability_linear(skill,score,linear_mean = linear_mean)
                hs_skill_linear = lambda accability, score : skill_linear(accability,score,linear_mean = linear_mean)
               
                for hs_aggregation_topscores_p in hyperparam_grid["aggregation_topscores_p"]:
                    print(f"aggregation_topscores_p = {hs_aggregation_topscores_p}")
                    for hs_max_iter in hyperparam_grid["max_iter"]:
                        print(f"max_iter = {hs_max_iter}")
                        for hs_finish_early in hyperparam_grid["finish_early"]:
                            print(f"finish_early = {hs_finish_early}")
                            for hs_error_change_prop in hyperparam_grid["error_change_prop"]:
                                print(f"error_change_prop = {hs_error_change_prop}")
                                
                                hs_crossvalidation_train_error_mean = 0
                                hs_crossvalidation_test_error_mean = 0                                
                                for hs_crossvalidation_i in range(crossvalidation_splits):
                                    print(f"Crossvalidation run {hs_crossvalidation_i}")                                          

                                    hs_scores_doubly_indexed_training_items, hs_scores_doubly_indexed_test_items = sklearn.model_selection.train_test_split(list(hs_scores_doubly_indexed.items()),test_size=test_set_size)
                                    hs_scores_doubly_indexed_training = dict(hs_scores_doubly_indexed_training_items)
                                    hs_scores_doubly_indexed_test = dict(hs_scores_doubly_indexed_test_items)

                                    hs_map_ratings = {map_id : default_rating for map_id in scores_by_map_id}
                                    hs_player_ratings = {player_id: default_rating for player_id in scores_by_player_id}

                                    hs_aggregation_fun = aggregation_topscores_f(hs_aggregation_topscores_p,default_rating)
                                    hs_error_aggregation_fun = aggregation_bottomscores_f(hs_error_aggregation_bottomscores_p,0)

                                    hs_bps = BiPartiteStabilizer(hs_map_ratings,hs_player_ratings,hs_scores_doubly_indexed_training,
                                                              hs_accability_linear,hs_skill_linear,hs_score_linear,
                                                              aggregation_fun=hs_aggregation_fun,
                                                              default_rating = default_rating,
                                                              error_aggregation_fun=hs_error_aggregation_fun,
                                                              max_iter=hs_max_iter,
                                                              finish_early=hs_finish_early,
                                                              error_change_prop=hs_error_change_prop)

                                    (hs_map_ratings,hs_player_ratings) = hs_bps.run()
                                    
                                    hs_error_train = hs_bps.average_error
                                    hs_error_test = hs_bps.measure_error(hs_scores_doubly_indexed_test)
                                    
                                    hs_crossvalidation_train_error_mean += hs_error_train/crossvalidation_splits
                                    hs_crossvalidation_test_error_mean += hs_error_test/crossvalidation_splits
                                    
                                hs_run = {
                                    "hyperparams":{
                                        "default_rating":default_rating,
                                        "truncexp_base_mean":truncexp_base_mean,
                                        "linear_mean":linear_mean,
                                        "test_set_size":test_set_size,
                                        "crosssvalidation_splits":crossvalidation_splits,
                                        "beta_alpha":hs_beta_alpha,
                                        "beta_beta":hs_beta_beta,
                                        "truncexp_max":hs_truncexp_max,
                                        "aggregation_topscores_p":hs_aggregation_topscores_p,
                                        "max_iter":hs_max_iter,
                                        "finish_early":hs_finish_early,
                                        "error_change_prop":hs_error_change_prop
                                    },
                                    "results":{
                                        "train_error":hs_crossvalidation_train_error_mean,
                                        "test_error":hs_crossvalidation_test_error_mean
                                    }
                                }
                                
                                pprint.pprint(hs_run,width=1)
                                
                                hs_runs_found = False
                                for hs_problem in hs_results:
                                    if hs_problem["problem_params"] == {"error_aggregation_bottomscores_p":hs_error_aggregation_bottomscores_p}:
                                        hs_problem["runs"].append(hs_run)
                                        hs_runs_found = True
                                        break

                                if not hs_runs_found:
                                    raise                                

24
error_aggregation_bottomscores_p = 0.25
beta-alpha = 10
beta-beta = 1.25
truncexp_max = 100
aggregation_topscores_p = 0.1
max_iter = 50
finish_early = True
error_change_prop = 0.005
Crossvalidation run 0
Iteration 1, Average error: 0.11139584189986264
Iteration 2, Average error: 0.07733602710121969
Iteration 3, Average error: 0.06976811070590316
Iteration 4, Average error: 0.0684619676952895
Iteration 5, Average error: 0.06893217913260781
Finishing early due to increased average error. Restoring previous values.
Crossvalidation run 1
Iteration 1, Average error: 0.1107590905937463
Iteration 2, Average error: 0.07726632880793681
Iteration 3, Average error: 0.06983346622229929
Iteration 4, Average error: 0.06863592714439146
Iteration 5, Average error: 0.06921761189919637
Finishing early due to increased average error. Restoring previous values.
Crossvalidation run 2
Iteration 1, Average error: 0.11252900070808251
Iteration 2, Average error: 0.07771281893341037
Iteration 3, Average erro

Crossvalidation run 2
Iteration 1, Average error: 0.11214835885896877
Iteration 2, Average error: 0.07777275385521165
Iteration 3, Average error: 0.07013970165392842
Iteration 4, Average error: 0.06882694725058713
Iteration 5, Average error: 0.0693610436571828
Iteration 6, Average error: 0.07028763098933025
Iteration 7, Average error: 0.07113842483002944
Iteration 8, Average error: 0.07185111381281756
Iteration 9, Average error: 0.07242722809095764
Iteration 10, Average error: 0.07286810888371846
Iteration 11, Average error: 0.0732038469274683
Finishing due to change in average error less than 0.005 (proportional)
Crossvalidation run 3
Iteration 1, Average error: 0.11303714010470162
Iteration 2, Average error: 0.07801905868012769
Iteration 3, Average error: 0.07006501152396925
Iteration 4, Average error: 0.06885783681736914
Iteration 5, Average error: 0.06943395574296106
Iteration 6, Average error: 0.07036835144462276
Iteration 7, Average error: 0.07120407546941732
Iteration 8, Average

Iteration 17, Average error: 0.07386656980882315
Iteration 18, Average error: 0.0738934454462424
Iteration 19, Average error: 0.0739097533321959
Iteration 20, Average error: 0.0739180127341475
Finishing due to change in average error less than 0.0002 (proportional)
Crossvalidation run 1
Iteration 1, Average error: 0.11159101934148442
Iteration 2, Average error: 0.07757726357864388
Iteration 3, Average error: 0.07003792216655377
Iteration 4, Average error: 0.06881288186086076
Iteration 5, Average error: 0.06938388706896599
Iteration 6, Average error: 0.07031328203396003
Iteration 7, Average error: 0.07115445766129298
Iteration 8, Average error: 0.07185227651153143
Iteration 9, Average error: 0.07240615485383488
Iteration 10, Average error: 0.07283688300176239
Iteration 11, Average error: 0.07316020670134485
Iteration 12, Average error: 0.07340137479529672
Iteration 13, Average error: 0.07358114679958264
Iteration 14, Average error: 0.07371357549514852
Iteration 15, Average error: 0.0738

Iteration 3, Average error: 0.04717887422374366
Iteration 4, Average error: 0.04708480492142447
Iteration 5, Average error: 0.047600390234583415
Finishing early due to increased average error. Restoring previous values.
Crossvalidation run 4
Iteration 1, Average error: 0.06932761068729835
Iteration 2, Average error: 0.049819543419106826
Iteration 3, Average error: 0.046659822007759436
Iteration 4, Average error: 0.046596104979924384
Iteration 5, Average error: 0.04716253366223445
Finishing early due to increased average error. Restoring previous values.
{'hyperparams': {'aggregation_topscores_p': 0.25,
                 'beta_alpha': 10,
                 'beta_beta': 1.25,
                 'crosssvalidation_splits': 5,
                 'default_rating': 10,
                 'error_change_prop': 0.001,
                 'finish_early': True,
                 'linear_mean': 10,
                 'max_iter': 50,
                 'test_set_size': 0.2,
                 'truncexp_base_mean': 10

Iteration 5, Average error: 0.04699336697221124
Iteration 6, Average error: 0.04754104670562957
Iteration 7, Average error: 0.04801120933437087
Iteration 8, Average error: 0.0483805057127089
Iteration 9, Average error: 0.0486679878194532
Iteration 10, Average error: 0.0488796592053799
Iteration 11, Average error: 0.049036705950502975
Iteration 12, Average error: 0.049152612040304536
Iteration 13, Average error: 0.04923840132956851
Iteration 14, Average error: 0.04930173239183083
Iteration 15, Average error: 0.04934822052502014
Finishing due to change in average error less than 0.001 (proportional)
Crossvalidation run 4
Iteration 1, Average error: 0.06971988381899477
Iteration 2, Average error: 0.04989320953948039
Iteration 3, Average error: 0.046619309377932495
Iteration 4, Average error: 0.046558663055798755
Iteration 5, Average error: 0.047071225297841586
Iteration 6, Average error: 0.04762777571180061
Iteration 7, Average error: 0.04811126231431548
Iteration 8, Average error: 0.0484

...

Iteration 4, Average error: 0.6111034034304291
Iteration 5, Average error: 0.6065576954372187
Iteration 6, Average error: 0.6041114764222859
Iteration 7, Average error: 0.6027036676593406
Iteration 8, Average error: 0.6018787348460752
Iteration 9, Average error: 0.6013832880857152
Finishing due to change in average error less than 0.001 (proportional)
Crossvalidation run 3
Iteration 1, Average error: 0.7227810689963488
Iteration 2, Average error: 0.6398308359803158
Iteration 3, Average error: 0.6171940289493772
Iteration 4, Average error: 0.6078513120037626
Iteration 5, Average error: 0.6032654403330092
Iteration 6, Average error: 0.6007622910940154
Iteration 7, Average error: 0.599327984700753
Iteration 8, Average error: 0.598472765173092
Iteration 9, Average error: 0.5979482227206592
Finishing due to change in average error less than 0.001 (proportional)
Crossvalidation run 4
Iteration 1, Average error: 0.72477988082557
Iteration 2, Average error: 0.640948806025649
Iteration 3, Avera

Iteration 1, Average error: 0.5013820601471902
Iteration 2, Average error: 0.44254789423796026
Iteration 3, Average error: 0.4285478920956223
Iteration 4, Average error: 0.42411121820192826
Iteration 5, Average error: 0.4226261320329094
Iteration 6, Average error: 0.4221577244206277
Iteration 7, Average error: 0.42206423184157577
Finishing due to change in average error less than 0.001 (proportional)
Crossvalidation run 1
Iteration 1, Average error: 0.5023221322729478
Iteration 2, Average error: 0.4432765718955763
Iteration 3, Average error: 0.42894395921974454
Iteration 4, Average error: 0.424268142519028
Iteration 5, Average error: 0.4226394278296644
Iteration 6, Average error: 0.4221019732206849
Iteration 7, Average error: 0.42196639040337675
Finishing due to change in average error less than 0.001 (proportional)
Crossvalidation run 2
Iteration 1, Average error: 0.5033913434289121
Iteration 2, Average error: 0.44467101717011465
Iteration 3, Average error: 0.43032142583080873
Iterati

Iteration 6, Average error: 0.4216333222630991
Iteration 7, Average error: 0.4214962179105222
Finishing due to change in average error less than 0.001 (proportional)
Crossvalidation run 1
Iteration 1, Average error: 0.5047174082474337
Iteration 2, Average error: 0.4457976622433636
Iteration 3, Average error: 0.431188693336882
Iteration 4, Average error: 0.42626073608150833
Iteration 5, Average error: 0.42446176301729327
Iteration 6, Average error: 0.42383840510590637
Iteration 7, Average error: 0.42365778004967214
Finishing due to change in average error less than 0.001 (proportional)
Crossvalidation run 2
Iteration 1, Average error: 0.5044055443109093
Iteration 2, Average error: 0.44493480986247763
Iteration 3, Average error: 0.4303145905050409
Iteration 4, Average error: 0.42530450270779663
Iteration 5, Average error: 0.4234224066586693
Iteration 6, Average error: 0.4227004222557999
Iteration 7, Average error: 0.4224342296674811
Finishing due to change in average error less than 0.00

Iteration 2, Average error: 0.34371726887979376
Iteration 3, Average error: 0.33476996704571865
Iteration 4, Average error: 0.33264385750168735
Iteration 5, Average error: 0.3322725780960675
Iteration 6, Average error: 0.3323313362939062
Finishing early due to increased average error. Restoring previous values.
Crossvalidation run 2
Iteration 1, Average error: 0.3865911697812834
Iteration 2, Average error: 0.34323950500211103
Iteration 3, Average error: 0.33427095183054223
Iteration 4, Average error: 0.33219446809002307
Iteration 5, Average error: 0.3318783240587948
Finishing due to change in average error less than 0.001 (proportional)
Crossvalidation run 3
Iteration 1, Average error: 0.3873199524835036
Iteration 2, Average error: 0.3445108716706943
Iteration 3, Average error: 0.33551894928124043
Iteration 4, Average error: 0.3332930069582639
Iteration 5, Average error: 0.3328716576554657
Iteration 6, Average error: 0.3329285938916736
Finishing early due to increased average error. Re

Crossvalidation run 4
Iteration 1, Average error: 0.3886386496103877
Iteration 2, Average error: 0.34499886478917996
Iteration 3, Average error: 0.33590858762315773
Iteration 4, Average error: 0.3337551928912526
Iteration 5, Average error: 0.3333983074249608
Iteration 6, Average error: 0.3334925067469373
Finishing due to change in average error less than 0.001 (proportional)
{'hyperparams': {'aggregation_topscores_p': 0.75,
                 'beta_alpha': 10,
                 'beta_beta': 1.25,
                 'crosssvalidation_splits': 5,
                 'default_rating': 10,
                 'error_change_prop': 0.001,
                 'finish_early': False,
                 'linear_mean': 10,
                 'max_iter': 50,
                 'test_set_size': 0.2,
                 'truncexp_base_mean': 10,
                 'truncexp_max': 100},
 'results': {'test_error': 0.5599319541694371,
             'train_error': 0.3329447645634497}}
error_change_prop = 0.0002
Crossvalidation r

In [34]:
pprint.pprint(hs_results,width=1)

[{'problem_params': {'error_aggregation_bottomscores_p': 0.25},
  'runs': [{'hyperparams': {'aggregation_topscores_p': 0.1,
                            'beta_alpha': 10,
                            'beta_beta': 1.25,
                            'crosssvalidation_splits': 5,
                            'default_rating': 10,
                            'error_change_prop': 0.005,
                            'finish_early': True,
                            'linear_mean': 10,
                            'max_iter': 50,
                            'test_set_size': 0.2,
                            'truncexp_base_mean': 10,
                            'truncexp_max': 100},
            'results': {'test_error': 0.04357049147673567,
                        'train_error': 0.06898890813794253}},
           {'hyperparams': {'aggregation_topscores_p': 0.1,
                            'beta_alpha': 10,
                            'beta_beta': 1.25,
                            'crosssvalidation_spli

In [36]:
with open('hyperparam_search_results.txt', 'w') as f:
    pprint.pprint(hs_results, width=1, stream=f)

In [38]:
for hs_problem in hs_results:
    hs_problem_params = hs_problem["problem_params"]
    hs_runs = hs_problem["runs"]
    
    min_test_error = math.inf
    min_test_error_hyperparams = None
    min_test_error_results = None
    
    for hs_run in hs_runs:
        hs_hyperparams = hs_run["hyperparams"]
        hs_run_results = hs_run["results"]
        hs_test_error = hs_run_results["test_error"]
        
        if hs_test_error < min_test_error:
            min_test_error = hs_test_error
            min_test_error_hyperparams = hs_hyperparams
            min_test_error_results = hs_run_results
    
    print(f"The run with the lowest validation error for problem {hs_problem_params} is the following:")
    pprint.pprint(min_test_error_hyperparams,width=1)
    pprint.pprint(min_test_error_results,width=1)

The best run for problem {'error_aggregation_bottomscores_p': 0.25} is the following:
{'aggregation_topscores_p': 0.25,
 'beta_alpha': 10,
 'beta_beta': 1.25,
 'crosssvalidation_splits': 5,
 'default_rating': 10,
 'error_change_prop': 0.005,
 'finish_early': True,
 'linear_mean': 10,
 'max_iter': 50,
 'test_set_size': 0.2,
 'truncexp_base_mean': 10,
 'truncexp_max': 100}
{'test_error': 0.04282308005007435,
 'train_error': 0.046902105002421285}
The best run for problem {'error_aggregation_bottomscores_p': 0.5} is the following:
{'aggregation_topscores_p': 0.5,
 'beta_alpha': 10,
 'beta_beta': 1.25,
 'crosssvalidation_splits': 5,
 'default_rating': 10,
 'error_change_prop': 0.001,
 'finish_early': True,
 'linear_mean': 10,
 'max_iter': 50,
 'test_set_size': 0.2,
 'truncexp_base_mean': 10,
 'truncexp_max': 100}
{'test_error': 0.1320926223129565,
 'train_error': 0.09698282943770359}
The best run for problem {'error_aggregation_bottomscores_p': 0.75} is the following:
{'aggregation_topscore

In [40]:
for hs_problem in hs_results:
    hs_problem_params = hs_problem["problem_params"]
    hs_runs = hs_problem["runs"]
    
    min_train_error = math.inf
    min_train_error_hyperparams = None
    min_train_error_results = None
    
    for hs_run in hs_runs:
        hs_hyperparams = hs_run["hyperparams"]
        hs_run_results = hs_run["results"]
        hs_train_error = hs_run_results["train_error"]
        
        if hs_train_error < min_train_error:
            min_train_error = hs_train_error
            min_train_error_hyperparams = hs_hyperparams
            min_train_error_results = hs_run_results
    
    print(f"The run with the lowest training error for problem {hs_problem_params} is the following:")
    pprint.pprint(min_train_error_hyperparams,width=1)
    pprint.pprint(min_train_error_results,width=1)

The run with the lowest training error for problem {'error_aggregation_bottomscores_p': 0.25} is the following:
{'aggregation_topscores_p': 0.75,
 'beta_alpha': 10,
 'beta_beta': 1.25,
 'crosssvalidation_splits': 5,
 'default_rating': 10,
 'error_change_prop': 0.0002,
 'finish_early': True,
 'linear_mean': 10,
 'max_iter': 50,
 'test_set_size': 0.2,
 'truncexp_base_mean': 10,
 'truncexp_max': 100}
{'test_error': 0.05113871120602363,
 'train_error': 0.036780493006173386}
The run with the lowest training error for problem {'error_aggregation_bottomscores_p': 0.5} is the following:
{'aggregation_topscores_p': 0.75,
 'beta_alpha': 10,
 'beta_beta': 1.25,
 'crosssvalidation_splits': 5,
 'default_rating': 10,
 'error_change_prop': 0.001,
 'finish_early': True,
 'linear_mean': 10,
 'max_iter': 50,
 'test_set_size': 0.2,
 'truncexp_base_mean': 10,
 'truncexp_max': 100}
{'test_error': 0.13999387053227116,
 'train_error': 0.08942626806536519}
The run with the lowest training error for problem {'

## Basic result exploration

In [44]:
map_outliers = {map_id : rating for (map_id,rating) in map_ratings.items() if rating < 3}
player_outliers = {player_id : rating for (player_id,rating) in player_ratings.items() if rating > 30}

In [45]:
print(map_outliers)
print(player_outliers)

{'3442cxxx91': 2.7259131538959065, '100991': 2.610672076032411, '3aa79xxxxxxxxx91': 1.9088994355352589, '2c803x91': 2.418268016886013, '194291': 2.980537503416201, '3b2bcxxxx91': 2.3496926998857606, '387a0xxxx91': 2.2651285980756257, '3b5a9xxxx92': 2.407735450155407, 'f58391': 2.8834790230234675, '41f391': 2.7982837800553537, '6b5f71': 2.7082330045353693}
{'3225556157461414': 30.38705527889929, '76561198180044686': 31.343970254062302, '1922350521131465': 35.00745902180232, '76561198988695829': 32.135924785631516, '76561199104169308': 30.546991749604217, '76561199085118735': 30.15188684391611, '76561198166061709': 34.001437783270426, '76561198404774259': 30.04852813818942, '76561198333869741': 31.646334111739314, '76561199465530115': 31.482582196513228}


In [49]:
hard_maps = {map_id : rating for (map_id,rating) in map_ratings.items() if rating < 2.5}
print(hard_maps)

{'3aa79xxxxxxxxx91': 1.9088994355352589, '2c803x91': 2.418268016886013, '3b2bcxxxx91': 2.3496926998857606, '387a0xxxx91': 2.2651285980756257, '3b5a9xxxx92': 2.407735450155407}


In [50]:
easy_maps = {map_id : rating for (map_id,rating) in map_ratings.items() if rating > 30}
print(easy_maps)

{'7e8f11': 32.11208460336793}


In [52]:
good_players = {player_id : rating for (player_id,rating) in player_ratings.items() if rating > 32}
print(good_players)

{'1922350521131465': 35.00745902180232, '76561198988695829': 32.135924785631516, '76561198166061709': 34.001437783270426}


In [62]:
#print(scores_doubly_indexed["7e8f11"])
#print(maps_by_id["7e8f11"])
print(players_by_id["76561198166061709"])

{'name': 'Taichidesu', 'country': 'JP', 'id': '76561198166061709', 'avatar': 'https://avatars.akamai.steamstatic.com/d2210eed27681acd8df596f6e11ecfe292d7de92_full.jpg'}


In [ ]:
print(player_ratings["76561199108348236"])

In [ ]:
print(scores_by_player_id["76561199108348236"])

In [ ]:
print(map_ratings["38419x92"])

## Useful stuff for extra detailed exploration

In [ ]:
def errors_in_map(map_id,map_ratings,player_ratings):
    result = {}
    map_rating = map_ratings[map_id]
    for player_id,score in scores_doubly_indexed[map_id].items():
        player_rating = player_ratings[player_id]
        calculated_w = bps.wfun(map_rating,player_rating)
        error = bps.error_fun(calculated_w,score)
        result[player_id] = {"calculated_w":calculated_w,"actual_score":score,"error":error}
        
    return result

In [ ]:
errors_this_map = errors_in_map("1442791",maps_1,players_1)
error_outliers = {player_id:error for (player_id,error) in errors_this_map.items() if error["error"] < 1}
print(error_outliers)

In [ ]:
scores_by_map_id["2c803x91"]["76561198279631500"]

In [ ]:
hard_maps = {map_id : rating for (map_id,rating) in maps_1.items() if rating < 2}
print(hard_maps)

{'a': {'b': 1,
       'c': {'d': 2,
             'e': 3},
       'f': 4},
 'g': {'h': 5,
       'i': 6}}
